In [ ]:
from qiskit import QuantumCircuit, transpile, IBMQ
from scheme_parameters import SchemeParameters
from decryption_circuit import create_decryption_circuit
from deletion_circuit import delete
from attack_circuit import breidbart_measure
from qiskit.providers.aer import AerSimulator
from qiskit.providers.fake_provider import FakeMontreal
import shutil
from datetime import datetime
import experiment
from typing import List, cast
from experiment_properties import ExperimentProperties
from experiment_manager import ExperimentManager
from qiskit.providers.ibmq.managed import IBMQJobManager
from qiskit.providers import JobStatus
import time

In [ ]:
# Experiment parameters
execution_datetime = datetime.now()
experiment_group_id = "experiment-manager-a"
system_string = "ibmq_manila" # Or insert other system name here
microsecond_delay = 0 # Delay between preparing the qubits and the first measurement, whether deletion or decryption
folder_prefix = "data"

# Backend parameters
provider = IBMQ.load_account()

# Ideal simulator
# backend = AerSimulator()

# Simulator with noise model
noise_model = FakeMontreal()
backend = AerSimulator.from_backend(noise_model)

# Real backend
backend = provider.get_backend("ibmq_manila")

# Qiskit parameters
optimization_level = 1
shots = 1000
qubits_per_circuit = 5

In [ ]:
experiment_manager = ExperimentManager()
ibmq_job_manager = IBMQJobManager()

experiment_properties = ExperimentProperties(
    execution_datetime=execution_datetime,
    shots=shots,
    system=system_string,
    microsecond_delay=microsecond_delay,
    optimization_level=optimization_level,
    qubits_per_circuit=qubits_per_circuit,
)

experiment_manager.set_up_experiments(
    experiment_properties,
    scheme_parameters=SchemeParameters.generate_from_lambda(1),
    num_experiments=15,
    folder_prefix=folder_prefix,
    base_id=experiment_group_id,
)

In [ ]:
for experiment in experiment_manager.experiments:
    deletion_circuit_test1 = delete(experiment.ciphertext)
    experiment.full_circuits.append(deletion_circuit_test1)

    decryption_circuit_test2 = create_decryption_circuit(experiment.key, experiment.ciphertext)
    experiment.full_circuits.append(decryption_circuit_test2)

    breidbart_circuit_test4 = breidbart_measure(experiment.ciphertext)
    experiment.full_circuits.append(breidbart_circuit_test4)

In [ ]:
raw_circuits = experiment_manager.collect_circuits()
transpiled_circuits = transpile(raw_circuits, backend=backend, optimization_level=optimization_level,
                                scheduling_method="alap" if microsecond_delay > 0 else None) # type: ignore
transpiled_circuits = cast(List[QuantumCircuit], transpiled_circuits)
experiment_manager.save_transpiled_circuits(transpiled_circuits)

job_set = ibmq_job_manager.run(transpiled_circuits, backend=backend, shots=shots, memory=True)

In [ ]:
while not all([status in (JobStatus.DONE, JobStatus.ERROR, JobStatus.CANCELLED) for status in job_set.statuses()]):
    print(f"Jobs completed: {job_set.statuses().count(JobStatus.DONE)}/{len(job_set.statuses())}")
    time.sleep(30)

print(f"Jobs completed: {job_set.statuses().count(JobStatus.DONE)}/{len(job_set.statuses())}")
experiment_manager.process_results(job_set.results())

In [ ]:
qubit_list = backend.properties().qubits if backend.properties() else None
for experiment in experiment_manager.experiments:
    experiment.export_to_folder(qubit_list=qubit_list)

In [ ]:
# Create zip file
shutil.make_archive("data_export", "zip", folder_prefix);